**Ответ на 11 билет**

Создайте и заполните базу данных сотрудников предприятия связи, содержащую следующие таблицы:
- Сотрудники (табельный номер, ФИО, пол, номер должности, отдел, дата рождения, стаж, семейное положение (Б-брак, Х- холост, Р- разведен), дети).
- Штатное расписание (номер должности, должность, оклад).

Составьте и выполните запросы на выборку отдела, фио, должности, зарплаты о тех сотрудниках, у которых зарплата больше средней, отсортировав по отделам, а внутри отдела по фамилии.


In [1]:
import sqlite3

# Подключение к базе данных (или её создание, если она не существует)
conn = sqlite3.connect('company.db')
c = conn.cursor()

# Создание таблицы Сотрудники
c.execute('''CREATE TABLE employees
             (id INTEGER PRIMARY KEY, full_name TEXT, gender TEXT, position_id INTEGER, department TEXT, 
             birth_date TEXT, experience INTEGER, marital_status TEXT, children INTEGER)''')

# Создание таблицы Штатное расписание
c.execute('''CREATE TABLE positions
             (position_id INTEGER PRIMARY KEY, position_name TEXT, salary REAL)''')

# Заполнение таблиц данными
employees_data = [(1, 'Иванов Иван Иванович', 'М', 1, 'Отдел разработки', '1985-04-12', 10, 'Б', 2),
                  (2, 'Петрова Мария Ивановна', 'Ж', 2, 'Отдел маркетинга', '1990-08-25', 5, 'Х', 0)]
positions_data = [(1, 'Разработчик', 100000),
                  (2, 'Маркетолог', 80000)]

c.executemany('INSERT INTO employees VALUES (?,?,?,?,?,?,?,?,?)', employees_data)
c.executemany('INSERT INTO positions VALUES (?,?,?)', positions_data)

conn.commit()

# Запрос на выборку отдела, ФИО, должности, зарплаты о тех сотрудниках, у которых зарплата больше средней
c.execute('''SELECT e.department, e.full_name, p.position_name, p.salary
             FROM employees e
             JOIN positions p ON e.position_id = p.position_id
             WHERE p.salary > (SELECT AVG(salary) FROM positions)
             ORDER BY e.department, e.full_name''')

# Вывод результатов запроса
for row in c.fetchall():
    print(row)

# Закрытие соединения с базой данных
conn.close()

('Отдел разработки', 'Иванов Иван Иванович', 'Разработчик', 100000.0)


Разработайте базу данных для предприятия связи, которая содержит следующие таблицы (не менее 3-х записей в таблицах):
-Сотрудники (табельный номер, ФИО, отдел, код должности, пол, дата рождения, стаж, семейное положение (Б-брак, Х- холост, Р- разведен), дети).
- Штатное расписание (код должности, должность, оклад).
Установите связь между таблицами.
Составьте запрос «Зарплата», в котором зарплата сотрудников вычисляется по формуле «оклад+премия». Премия зависит от стажа, если стаж <=5 лет премия равна 50 % от оклада; если стаж больше 5 лет премия равна 100 % от оклада.
 Составьте запрос «Отделы»: количество сотрудников, максимальная зарплата, минимальная зарплата, общее количество детей в отделе


In [ ]:
import sqlite3

# Создание соединения с базой данных
conn = sqlite3.connect('enterprise.db')
cursor = conn.cursor()

# Создание таблицы "Сотрудники"
cursor.execute('''CREATE TABLE IF NOT EXISTS Employees (
                employee_id INTEGER PRIMARY KEY,
                full_name TEXT,
                department TEXT,
                position_code TEXT,
                gender TEXT,
                birth_date TEXT,
                experience INTEGER,
                marital_status TEXT,
                children INTEGER
                )''')

# Создание таблицы "Штатное расписание"
cursor.execute('''CREATE TABLE IF NOT EXISTS StaffSchedule (
                position_code TEXT PRIMARY KEY,
                position TEXT,
                salary INTEGER
                )''')

# Вставка данных в таблицу "Сотрудники"
employees_data = [
    (1, 'Иванов Иван Иванович', 'Отдел продаж', 'P001', 'М', '1990-05-15', 7, 'Б', 2),
    (2, 'Петрова Анна Сергеевна', 'Отдел маркетинга', 'P002', 'Ж', '1985-10-20', 3, 'Х', 0),
    (3, 'Сидоров Павел Владимирович', 'Отдел разработки', 'P003', 'М', '1982-12-10', 10, 'Р', 1)
]

cursor.executemany('''INSERT INTO Employees VALUES (?,?,?,?,?,?,?,?,?)''', employees_data)

# Вставка данных в таблицу "Штатное расписание"
schedule_data = [
    ('P001', 'Менеджер по продажам', 50000),
    ('P002', 'Маркетолог', 60000),
    ('P003', 'Программист', 70000)
]

cursor.executemany('''INSERT INTO StaffSchedule VALUES (?,?,?)''', schedule_data)

# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()

In [ ]:
# Создание соединения с базой данных
conn = sqlite3.connect('enterprise.db')
cursor = conn.cursor()

# Запрос "Зарплата"
cursor.execute('''SELECT e.full_name, s.salary + (CASE WHEN e.experience <= 5 THEN s.salary * 0.5 ELSE s.salary END) AS total_salary
                FROM Employees e
                JOIN StaffSchedule s ON e.position_code = s.position_code''')

# Вывод результатов запроса

table = {col[0]:[] for col in cursor.description}


for row in cursor.fetchall():
    for i, col in enumerate(table.keys()):
        table[col].append(row[i])

# Закрытие соединения
conn.close()
print(table)

In [ ]:
# Создание соединения с базой данных
conn = sqlite3.connect('enterprise.db')
cursor = conn.cursor()

# Запрос "Отделы"
cursor.execute('''SELECT department, COUNT(employee_id) AS num_employees, MAX(salary) AS max_salary, MIN(salary) AS min_salary, SUM(children) AS total_children
                FROM Employees e
                JOIN StaffSchedule s ON e.position_code = s.position_code
                GROUP BY department''')

# Вывод результатов запроса
table = {col[0]:[] for col in cursor.description}

for row in cursor.fetchall():
    for i, col in enumerate(table.keys()):
        table[col].append(row[i])

# Закрытие соединения
conn.close()
print(table)

Создайте и заполните базу данных сотрудников предприятия связи: - Сотрудники (табельный номер, ФИО, отдел, код должности, пол, дата рождения, стаж, семейное положение (Б-брак, Х- холост, Р- разведен), дети).
- Штатное расписание (код должности, должность, оклад),
Установите связь между таблицами.
 Составьте запросы для выборки информации:
- о сотрудниках: подразделение, ФИО, зарплата в рублях, зарплата в долларах (вычислить), отсортировав по фамилии (зарплата сотрудников вычисляется по формуле «оклад + премия», где премия равна 80 % от оклада.);
- по каждому подразделению вычислить: среднюю заработную плату, количество женщин и детей.


In [1]:
import sqlite3

# Подключение к базе данных
conn = sqlite3.connect('enterprise.db')
cursor = conn.cursor()

# Создание таблиц
cursor.execute('''
CREATE TABLE IF NOT EXISTS Сотрудники (
    табельный_номер INTEGER PRIMARY KEY,
    ФИО TEXT NOT NULL,
    отдел TEXT NOT NULL,
    код_должности INTEGER NOT NULL,
    пол TEXT NOT NULL,
    дата_рождения TEXT NOT NULL,
    стаж INTEGER NOT NULL,
    семейное_положение TEXT NOT NULL,
    дети INTEGER NOT NULL,
    FOREIGN KEY (код_должности) REFERENCES Штатное_расписание (код_должности)
)
''')

cursor.execute('''
CREATE TABLE IF NOT EXISTS Штатное_расписание (
    код_должности INTEGER PRIMARY KEY,
    должность TEXT NOT NULL,
    оклад REAL NOT NULL
)
''')

# Заполнение таблиц данными
# Пример данных для таблицы Сотрудники
cursor.execute('''
INSERT INTO Сотрудники (табельный_номер, ФИО, отдел, код_должности, пол, дата_рождения, стаж, семейное_положение, дети)
VALUES
(1, 'Иванов Иван Иванович', 'Отдел разработки', 101, 'М', '1980-01-01', 10, 'Б', 2),
(2, 'Петрова Мария Ивановна', 'Отдел маркетинга', 102, 'Ж', '1985-02-02', 8, 'Х', 0),
(3, 'Иванова Мария Петровна', 'Отдел маркетинга', 102, 'Ж', '1985-02-02', 8, 'Х', 0)
''')

# Пример данных для таблицы Штатное_расписание
cursor.execute('''
INSERT INTO Штатное_расписание (код_должности, должность, оклад)
VALUES
(101, 'Разработчик', 60000),
(102, 'Маркетолог', 50000)
''')

# Сохранение изменений
conn.commit()

# Запросы для выборки информации
# Информация о сотрудниках
cursor.execute('''
SELECT С.отдел, С.ФИО, Ш.оклад + Ш.оклад * 0.8 AS зарплата_руб, (Ш.оклад + Ш.оклад * 0.8) / 90 AS зарплата_долл
FROM Сотрудники С
JOIN Штатное_расписание Ш ON С.код_должности = Ш.код_должности
ORDER BY С.ФИО
''')

columns = tuple(col[0] for col in cursor.description)
employees_info = cursor.fetchall()

print(columns)
for employee in employees_info:
    print(employee)
print('\n\n', "*"*100, "\n\n")

# Для вычисления зарплаты в долларах необходимо знать текущий курс. Предположим, что Курс_доллара = 60
# Замените Курс_доллара на актуальное значение при выполнении запроса

# Информация по каждому подразделению
cursor.execute('''
SELECT С.отдел, AVG(Ш.оклад + Ш.оклад * 0.8) AS средняя_зарплата, SUM(CASE WHEN С.пол = 'Ж' THEN 1 ELSE 0 END) AS количество_женщин, SUM(С.дети) AS количество_детей
FROM Сотрудники С
JOIN Штатное_расписание Ш ON С.код_должности = Ш.код_должности
GROUP BY С.отдел
''')

# Получение результатов
columns = tuple(col[0] for col in cursor.description)
employees_info = cursor.fetchall()

print(columns)
for employee in employees_info:
    print(employee)
print('\n\n', "*"*100, "\n\n")
# Закрытие соединения
conn.close()

('отдел', 'ФИО', 'зарплата_руб', 'зарплата_долл')
('Отдел разработки', 'Иванов Иван Иванович', 108000.0, 1200.0)
('Отдел маркетинга', 'Иванова Мария Петровна', 90000.0, 1000.0)
('Отдел маркетинга', 'Петрова Мария Ивановна', 90000.0, 1000.0)


 **************************************************************************************************** 


('отдел', 'средняя_зарплата', 'количество_женщин', 'количество_детей')
('Отдел маркетинга', 90000.0, 2, 0)
('Отдел разработки', 108000.0, 0, 2)


 **************************************************************************************************** 




Создайте и заполните базу данных на Python сотрудников предприятия связи, содержащей следующие таблицы:
-Сотрудники (табельный номер, фио, пол, номер_должности, отдел, дата рождения, хобби, стаж, семейное положение (Б-брак, Х- холост, Р- разведен), дети).
- Штатное расписание (номер_должности, должность, оклад).
Установите связь между таблицами.
Создайте следующие запросы
- «Подарки» - сколько подарков на 8 Марта нужно приготовить в каждом отделе.

In [1]:
import sqlite3

# Подключение к базе данных (или ее создание, если она не существует)
conn = sqlite3.connect('enterprise.db')
cursor = conn.cursor()

# Создание таблицы Сотрудники
cursor.execute('''
CREATE TABLE IF NOT EXISTS Сотрудники (
    табельный_номер INTEGER PRIMARY KEY,
    фио TEXT NOT NULL,
    пол TEXT CHECK(пол IN ('М', 'Ж')),
    номер_должности INTEGER,
    отдел TEXT,
    дата_рождения DATE,
    хобби TEXT,
    стаж INTEGER,
    семейное_положение TEXT CHECK(семейное_положение IN ('Б', 'Х', 'Р')),
    дети INTEGER,
    FOREIGN KEY (номер_должности) REFERENCES Штатное_расписание (номер_должности)
)
''')


# Создание таблицы Штатное расписание
cursor.execute('''
CREATE TABLE IF NOT EXISTS Штатное_расписание (
    номер_должности INTEGER PRIMARY KEY,
    должность TEXT NOT NULL,
    оклад DECIMAL NOT NULL
)
''')


cursor.execute('''
INSERT INTO Сотрудники (табельный_номер, фио, пол, номер_должности, отдел, дата_рождения, хобби, стаж, семейное_положение, дети)
VALUES
(1, 'Иванов Иван Иванович', 'М',  101, 'Отдел разработки', '1980-01-01', 'работать',  10, 'Б', 2),
(2, 'Иванов Жорик Иванович', 'М',  102, 'Отдел безделии', '1980-01-01', 'работать',  5, 'Б', 2),
(3, 'Иванов Ибрагим Иванович', 'Ж',  102, 'Отдел безделии', '1980-01-01', 'работать',  3, 'Б', 0)
''')


# Пример данных для таблицы Штатное_расписание
cursor.execute('''
INSERT INTO Штатное_расписание (номер_должности, должность, оклад)
VALUES
(101, 'Разработчик', 60000),
(102, 'Маркетолог', 50000)
''')



# Запрос "Подарки" - подсчет подарков на 8 Марта в каждом отделе
query = '''
SELECT отдел, COUNT(*) as количество_подарков
FROM Сотрудники
WHERE пол = 'Ж'
GROUP BY отдел
'''

# Выполнение запроса
cursor.execute(query)
gifts = cursor.fetchall()
for department, count in gifts:
    print(f'В отделе "{department}" нужно приготовить {count} подарков на 8 Марта.')

# Закрытие соединения с базой данных
conn.commit()
conn.close()

В отделе "Отдел безделии" нужно приготовить 1 подарков на 8 Марта.


Спроектируйте базу данных для предприятия связи, содержащую следующие таблицы:
- Сотрудники (отдел, табельный номер, фио, пол, код должности, дата рождения, хобби, стаж).
- Штатное расписание (код должности, должность, оклад).
Установите связь между таблицами.

Создайте формы(у) для ввода и редактирования данных в таблицах.

Сформируйте следующие запросы:
- «Зарплата»: информация о зарплате каждого сотрудника, вычисленная по формуле зарплата = оклад + премия, где премия зависит от стажа работы и если стаж меньше или равен 5 лет, то премия равна 50 % от оклада; если стаж больше 5 лет, премия равна 100 % от оклада;
-«сотрудницы»: (пол- женский), имеющих зарплату выше средней по отделу, вывести поля: отдел, фио, пол, должность, зарплата.


In [1]:
import sqlite3

# Подключение к базе данных (или её создание, если она не существует)
conn = sqlite3.connect('company_database.db')
cursor = conn.cursor()

# Создание таблицы сотрудников
cursor.execute('''
CREATE TABLE IF NOT EXISTS employees (
    department TEXT,
    employee_id INTEGER PRIMARY KEY,
    full_name TEXT,
    gender TEXT,
    position_code INTEGER,
    birth_date TEXT,
    hobbies TEXT,
    experience INTEGER
)
''')

# Создание таблицы штатного расписания
cursor.execute('''
CREATE TABLE IF NOT EXISTS positions (
    position_code INTEGER PRIMARY KEY,
    position_name TEXT,
    salary REAL
)
''')

# Сохранение изменений и закрытие соединения с базой данных
conn.commit()
conn.close()

In [4]:
import tkinter as tk
from tkinter import messagebox

def insert_employee():
    # Здесь должен быть код для вставки данных сотрудника в базу данных
    # Пример:
    conn = sqlite3.connect('company_database.db')
    cursor = conn.cursor()
    cursor.execute('''INSERT INTO employees (department, employee_id, full_name, gender, position_code, birth_date, hobbies, experience) VALUES (?, ?, ?, ?, ?, ?, ?, ?)''', (department_entry.get(), employee_id_entry.get(), full_name_entry.get(), gender_entry.get(), position_code_entry.get(), birth_date_entry.get(), hobbies_entry.get(), experience_entry.get()))
    conn.commit()
    conn.close()
    messagebox.showinfo("Успех", "Сотрудник добавлен")
    
def edit_employee():
    # Здесь должен быть код для редактирования данных сотрудника в базе данных
    # Пример:
    conn = sqlite3.connect('company_database.db')
    cursor = conn.cursor()
    cursor.execute('''UPDATE employees SET department=?, full_name=?, gender=?, position_code=?, birth_date=?, hobbies=?, experience=? WHERE employee_id=?''', (department_entry.get(), full_name_entry.get(), gender_entry.get(), position_code_entry.get(), birth_date_entry.get(), hobbies_entry.get(), experience_entry.get(), employee_id_entry.get()))
    conn.commit()
    conn.close()
    messagebox.showinfo("Успех", "Данные сотрудника обновлены")

# Создание основного окна
root = tk.Tk()
root.title("Управление базой данных сотрудников")

# Создание формы для ввода данных сотрудника
employee_frame = tk.LabelFrame(root, text="Сотрудник")
employee_frame.grid(row=0, column=0, padx=10, pady=10, sticky="ew")

# Поля ввода
labels = ['Отдел', 'Табельный номер', 'ФИО', 'Пол', 'Код должности', 'Дата рождения', 'Хобби', 'Стаж']
for i, label in enumerate(labels):
    tk.Label(employee_frame, text=label).grid(row=i, column=0, sticky="e", padx=5, pady=5)

department_entry = tk.Entry(employee_frame)
employee_id_entry = tk.Entry(employee_frame)
full_name_entry = tk.Entry(employee_frame)
gender_entry = tk.Entry(employee_frame)
position_code_entry = tk.Entry(employee_frame)
birth_date_entry = tk.Entry(employee_frame)
hobbies_entry = tk.Entry(employee_frame)
experience_entry = tk.Entry(employee_frame)

entries = [department_entry, employee_id_entry, full_name_entry, gender_entry, position_code_entry, birth_date_entry, hobbies_entry, experience_entry]
for i, entry in enumerate(entries):
    entry.grid(row=i, column=1, sticky="ew", padx=5, pady=5)

# Кнопки
tk.Button(employee_frame, text="Добавить сотрудника", command=insert_employee).grid(row=len(labels), column=0, padx=5, pady=5, sticky="ew")
tk.Button(employee_frame, text="Редактировать сотрудника", command=edit_employee).grid(row=len(labels), column=1, padx=5, pady=5, sticky="ew")

# Запуск основного цикла Tkinter
root.mainloop()

In [ ]:
def calculate_salary():
    # Запрос на расчет зарплаты с учетом премии в зависимости от стажа
    cursor.execute('''
    SELECT e.full_name, p.salary, e.experience,
    CASE
        WHEN e.experience <= 5 THEN p.salary * 1.5
        WHEN e.experience > 5 THEN p.salary * 2.0
    END AS total_salary
    FROM employees e
    JOIN positions p ON e.position_code = p.position_code
    ''')
    return cursor.fetchall()

def get_female_employees_with_higher_salary():
    # Запрос на получение сотрудниц с зарплатой выше средней по отделу
    cursor.execute('''
    SELECT e.department, e.full_name, e.gender, p.position_name,
    (p.salary + CASE WHEN e.experience <= 5 THEN p.salary * 0.5 ELSE p.salary END) AS total_salary
    FROM employees e
    JOIN positions p ON e.position_code = p.position_code
    WHERE e.gender = 'женский' AND total_salary > (
        SELECT AVG(p.salary + CASE WHEN e.experience <= 5 THEN p.salary * 0.5 ELSE p.salary END)
        FROM employees e
        JOIN positions p ON e.position_code = p.position_code
        WHERE e.department = e.department
    )
    ''')
    return cursor.fetchall()



calculate_salary()

Создайте и заполните базу данных на Python сотрудников предприятия связи, содержащую следующие таблицы:
- Сотрудники (табельный номер, ФИО, пол, номер должности, отдел, дата рождения, стаж, семейное положение (Б-брак, Х- холост, Р- разведен), дети).
- Штатное расписание (номер должности, должность, оклад).

Установите связь между таблицами.

Составьте запрос «Зарплата», в котором зарплата сотрудников вычисляется по формуле «оклад + премия». Премия зависит от стажа и составляет:
- 50 % от оклада, если стаж меньше или равен 3годам;
- 70 % от оклада, если стаж больше 3, но меньше 8;
- 100 % от оклада в остальных случаях.

In [1]:
import sqlite3

# Подключение к базе данных (или ее создание, если она не существует)
conn = sqlite3.connect('enterprise.db')
cursor = conn.cursor()

# Создание таблицы Сотрудники
cursor.execute('''
CREATE TABLE IF NOT EXISTS Сотрудники (
    табельный_номер INTEGER PRIMARY KEY,
    ФИО TEXT NOT NULL,
    пол TEXT CHECK(пол IN ('М', 'Ж')),
    номер_должности INTEGER,
    отдел TEXT,
    дата_рождения DATE,
    стаж INTEGER,
    семейное_положение TEXT CHECK(семейное_положение IN ('Б', 'Х', 'Р')),
    дети INTEGER,
    FOREIGN KEY (номер_должности) REFERENCES Штатное_расписание (номер_должности)
)
''')

# Создание таблицы Штатное расписание
cursor.execute('''
CREATE TABLE IF NOT EXISTS Штатное_расписание (
    номер_должности INTEGER PRIMARY KEY,
    должность TEXT NOT NULL,
    оклад REAL NOT NULL
)
''')

# Сохранение изменений
conn.commit()


# Заполнение таблицы Штатное_расписание
positions = [
    (1, 'Инженер', 40000),
    (2, 'Бухгалтер', 35000),
    (3, 'Менеджер', 45000),
    (4, 'Директор', 80000),
]

cursor.executemany('INSERT INTO Штатное_расписание VALUES (?, ?, ?)', positions)
conn.commit()

# Заполнение таблицы Сотрудники
employees = [
    (101, 'Иванов Иван Иванович', 'М', 1, 'Отдел разработки', '1985-06-15', 5, 'Б', 2),
    (102, 'Петрова Мария Ивановна', 'Ж', 2, 'Бухгалтерия', '1990-03-22', 2, 'Х', 0),
    (103, 'Сидоров Алексей Петрович', 'М', 3, 'Отдел продаж', '1978-11-01', 10, 'Р', 1),
    (104, 'Кузнецова Елена Сергеевна', 'Ж', 1, 'Отдел разработки', '1988-08-19', 7, 'Б', 1),
    (105, 'Александров Сергей Михайлович', 'М', 4, 'Управление', '1970-01-05', 15, 'Х', 3),
]

cursor.executemany('INSERT INTO Сотрудники VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)', employees)
conn.commit()

# Запрос Зарплата

cursor.execute('''
SELECT С.ФИО, Ш.оклад,
CASE
    WHEN С.стаж <= 3 THEN Ш.оклад * 1.5
    WHEN С.стаж > 3 AND С.стаж < 8 THEN Ш.оклад * 1.7
    ELSE Ш.оклад * 2.0
END AS Зарплата
FROM Сотрудники С
JOIN Штатное_расписание Ш ON С.номер_должности = Ш.номер_должности
''')

columns = tuple(col[0] for col in cursor.description)

# Вывод результатов запроса Зарплата
print(columns)
for row in cursor.fetchall():
    print(row)

# Закрытие соединения с базой данных
conn.close()

('ФИО', 'оклад', 'Зарплата')
('Иванов Иван Иванович', 40000.0, 68000.0)
('Петрова Мария Ивановна', 35000.0, 52500.0)
('Сидоров Алексей Петрович', 45000.0, 90000.0)
('Кузнецова Елена Сергеевна', 40000.0, 68000.0)
('Александров Сергей Михайлович', 80000.0, 160000.0)


In [5]:
import math

def factorial_n(n):
    f = 1

    for i in range(1, n+1):
        f = f * i
    return f


factorial_n(4), math.factorial(4)

(24, 24)

In [8]:
class Train:
    def __init__(self, destination, train_number, departure_time):
        self._destination = destination
        self._train_number = train_number
        self._departure_time = departure_time


    @property
    def destination(self):
        return self._destination


    @property
    def train_number(self):
        return self._train_number


    @property
    def departure_time(self):
        return self._departure_time


class Station:
    def __init__(self):
        self.trains = []


    def add_train(self, train):
        self.trains.append(train)


    def get_train_by_number(self, number):
        for train in self.trains:
            if train.train_number == number:
                return (train.train_number,train.destination, train.departure_time)
        return None


    def get_trains_after_time(self, time):
        return [(train.train_number,train.destination, train.departure_time) for train in self.trains if train.departure_time > time]


    def compare_departure_times(self, train1, train2):
        if train1.departure_time < train2.departure_time:
            return f"{train1.departure_time}"+" < "+f"{train2.departure_time}"
        else: return f"{train1.departure_time}"+" > "+f"{train2.departure_time}"


    def get_trains_to_destination(self, destination):
        trains_list = []
        # sorted_list = sorted([train for train in self.trains if train.destination == destination], key=lambda x: x.departure_time)
        sorted_list = sorted(self.trains, key=lambda x: x.departure_time)
        for i in sorted_list:
            if i.destination == destination: 
                trains_list.append(i)
        return [(train.train_number,train.destination, train.departure_time) for train in trains_list]
   
# Creating instances of Train
train1 = Train("Moscow", "123A", "08:00")
train2 = Train("Saint Petersburg", "456B", "09:30")
train3 = Train("Kiev", "789C", "10:15")
train4 = Train("Moscow", "123B", "09:00")


# Creating an instance of Station and adding trains to it
station = Station()
station.add_train(train1)
station.add_train(train2)
station.add_train(train3)
station.add_train(train4)


print(station.trains)


# Demonstrating functionality
while True:
    cmd = ""
    print('''
          1: вывод информации о поезде по номеру с помощью индекса;
          2: вывод информации о поездах, отправляющихся после введенного с клавиатуры времени;
          3: вывод информации о сравнение времени отправления двух поездов;
          4: вывод информации о поездах, отправляющихся в заданный пункт назначения;
          0: выход
          '''
    )
    cmd = input('введите команду: ')
    match cmd:
        case "1":
            print(station.get_train_by_number(input("Номер поезда: ")))
        case "2":
            print(station.get_trains_after_time(input("Время(чч:мм): ")))
        case "3":
            print(station.compare_departure_times(train1, train2))
        case "4":
            print(station.get_trains_to_destination(input("Место назначения: ")))
        case "0": break
        case _:
            print("Code not found")


print(station.get_train_by_number("123A"))
print(station.get_trains_after_time("09:00"))
print(station.compare_departure_times(train1, train2))
print(station.get_trains_to_destination("Moscow"))


[<__main__.Train object at 0x000001AEEC98BD30>, <__main__.Train object at 0x000001AEEC98BCD0>, <__main__.Train object at 0x000001AEEC98BE50>, <__main__.Train object at 0x000001AEEC975540>]

          1: вывод информации о поезде по номеру с помощью индекса;
          2: вывод информации о поездах, отправляющихся после введенного с клавиатуры времени;
          3: вывод информации о сравнение времени отправления двух поездов;
          4: вывод информации о поездах, отправляющихся в заданный пункт назначения;
          0: выход
          
('123A', 'Moscow', '08:00')

          1: вывод информации о поезде по номеру с помощью индекса;
          2: вывод информации о поездах, отправляющихся после введенного с клавиатуры времени;
          3: вывод информации о сравнение времени отправления двух поездов;
          4: вывод информации о поездах, отправляющихся в заданный пункт назначения;
          0: выход
          
('123A', 'Moscow', '08:00')
[('456B', 'Saint Petersburg', '09:30'), ('789

Создайте и заполните базу данных учёта ремонтных работ линий связи, содержащую следующие таблицы
-  Работники (табельный номер, фио, пол, должность, дата рождения, хобби, стаж, номер бригады).
-  Аварии (дата аварии, время аварии, место аварии, время устранения, номер бригады, причина).
Создайте запрос, в котором пользователь   вводит с клавиатуры дату и запрос выводит информацию, какие аварии были в этот день и бригады, устранявшие их.
Сформируйте запрос на вывод информации об авариях, произошедших в определенный период (задать с клавиатуры).


In [9]:
import sqlite3
from datetime import datetime

# Создание и подключение к базе данных
conn = sqlite3.connect('repair_works.db')
cursor = conn.cursor()

# Создание таблицы Работники
cursor.execute('''
CREATE TABLE IF NOT EXISTS Workers (
    worker_id INTEGER PRIMARY KEY,
    full_name TEXT,
    gender TEXT,
    position TEXT,
    birth_date TEXT,
    hobby TEXT,
    experience INTEGER,
    brigade_number INTEGER
)
''')

# Создание таблицы Аварии
cursor.execute('''
CREATE TABLE IF NOT EXISTS Accidents (
    accident_id INTEGER PRIMARY KEY,
    accident_date TEXT,
    accident_time TEXT,
    accident_place TEXT,
    repair_time TEXT,
    brigade_number INTEGER,
    reason TEXT
)
''')

# Пример заполнения таблицы Работники
workers = [
    (1, 'Иванов Иван Иванович', 'М', 'Инженер', '1980-01-01', 'Рыбалка', 10, 1),
    (2, 'Петров Петр Петрович', 'М', 'Техник', '1985-02-02', 'Футбол', 5, 2),
    (3, 'Сидорова Анна Ивановна', 'Ж', 'Мастер', '1990-03-03', 'Чтение', 8, 1),
    (4, 'Кузнецов Алексей Сергеевич', 'М', 'Электрик', '1988-04-04', 'Велоспорт', 7, 3),
    (5, 'Морозова Елена Викторовна', 'Ж', 'Инженер', '1992-05-05', 'Плавание', 6, 2),
]

cursor.executemany('''
INSERT INTO Workers (worker_id, full_name, gender, position, birth_date, hobby, experience, brigade_number)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
''', workers)

# Пример заполнения таблицы Аварии
accidents = [
    (1, '2024-05-25', '14:30', 'Улица Ленина', '16:00', 1, 'Перегрузка линии'),
    (2, '2024-05-25', '18:00', 'Проспект Мира', '12:00', 2, 'Повреждение кабеля'),
    (3, '2024-05-26', '09:00', 'Улица Пушкина', '11:30', 1, 'Короткое замыкание'),
    (4, '2024-05-28', '13:00', 'Улица Гагарина', '15:00', 3, 'Обрыв провода'),
    (5, '2024-05-29', '08:00', 'Улица Советская', '10:30', 2, 'Перегрузка линии'),
]

cursor.executemany('''
INSERT INTO Accidents (accident_id, accident_date, accident_time, accident_place, repair_time, brigade_number, reason)
VALUES (?, ?, ?, ?, ?, ?, ?)
''', accidents)

conn.commit()


# Функция для получения аварий по дате и времени
def get_accidents_by_date_and_time(date, start_time, end_time):
    cursor.execute('''
    SELECT accident_date, accident_time, accident_place, repair_time, brigade_number, reason
    FROM Accidents
    WHERE accident_date = ? AND accident_time BETWEEN ? AND ?
    ''', (date, start_time, end_time))
    return cursor.fetchall()


# Функция для получения аварий за период с учетом времени
def get_accidents_by_period_with_time(start_datetime, end_datetime):
    cursor.execute('''
    SELECT accident_date, accident_time, accident_place, repair_time, brigade_number, reason
    FROM Accidents
    WHERE datetime(accident_date || ' ' || accident_time) BETWEEN ? AND ?
    ''', (start_datetime, end_datetime))
    return cursor.fetchall()


# Ввод даты и времени с клавиатуры и вывод информации об авариях в этот день и время
date = input("Введите дату (YYYY-MM-DD): ")
start_time = input("Введите начальное время (HH:MM): ")
end_time = input("Введите конечное время (HH:MM): ")
accidents_on_date_and_time = get_accidents_by_date_and_time(date, start_time, end_time)
print(f"Аварии на {date} с {start_time} по {end_time}:")
for accident in accidents_on_date_and_time:
    print(accident)

# Ввод начальной и конечной даты и времени с клавиатуры и вывод информации об авариях за этот период
start_datetime = input("Введите начальную дату и время периода (YYYY-MM-DD HH:MM): ")
end_datetime = input("Введите конечную дату и время периода (YYYY-MM-DD HH:MM): ")
accidents_in_period_with_time = get_accidents_by_period_with_time(start_datetime, end_datetime)
print(f"Аварии с {start_datetime} по {end_datetime}:")
for accident in accidents_in_period_with_time:
    print(accident)

# Закрытие соединения с базой данных
conn.close()

Аварии на 2024-05-25 с 12:00 по 20:30:
('2024-05-25', '14:30', 'Улица Ленина', '16:00', 1, 'Перегрузка линии')
('2024-05-25', '18:00', 'Проспект Мира', '12:00', 2, 'Повреждение кабеля')
Аварии с 2024-05-25 14:30 по 2024-05-28 13:00:
('2024-05-25', '14:30', 'Улица Ленина', '16:00', 1, 'Перегрузка линии')
('2024-05-25', '18:00', 'Проспект Мира', '12:00', 2, 'Повреждение кабеля')
('2024-05-26', '09:00', 'Улица Пушкина', '11:30', 1, 'Короткое замыкание')
